# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

Для начала иследуем то, что нам досталось от заказчика. Исследуем данные на наличие ошибок, пропусков и прочих проблемок.

In [1]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
pd.options.display.float_format ='{:,.3f}'.format


data = pd.read_csv('/datasets/data.csv')

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"-8,437.673",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.639",покупка жилья
1,1,"-4,024.804",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.014",приобретение автомобиля
2,0,"-5,623.423",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.952",покупка жилья
3,3,"-4,124.747",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.550",дополнительное образование
4,0,"340,266.072",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.078",сыграть свадьбу
5,0,-926.186,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.565",покупка жилья
6,0,"-2,879.202",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.972",операции с жильем
7,0,-152.780,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.934",образование
8,2,"-6,929.865",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.832",на проведение свадьбы
9,0,"-2,188.756",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.938",покупка жилья для семьи


In [3]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,"21,525.000","19,351.000","21,525.000","21,525.000","21,525.000","21,525.000","19,351.000"
mean,0.539,"63,046.498",43.293,0.817,0.973,0.081,"167,422.302"
std,1.382,"140,827.312",12.575,0.548,1.420,0.273,"102,971.566"
min,-1.000,"-18,388.950",0.000,0.000,0.000,0.000,"20,667.264"
25%,0.000,"-2,747.424",33.000,1.000,0.000,0.000,"103,053.153"
50%,0.000,"-1,203.370",42.000,1.000,0.000,0.000,"145,017.938"
75%,1.000,-291.096,53.000,1.000,1.000,0.000,"203,435.068"
max,20.000,"401,755.400",75.000,4.000,4.000,1.000,"2,265,604.029"


Возникли первые вопросы: 
1. есть пропуски в столбцах days_employed и total_income, причем похоже в одних и тех же строках
2. проблемы с регистром в education
3. отрицательные значения в столбце days_employed, а так же средний стаж 63046 дней - почти 173 года. Роботы, не иначе. Стоит уточнить у источника информации что они имели ввиду.
4. Кто-то потерял ребенка, а может взял в долг) (значение -1 в столбце children), а максимальное значение 20 - тоже возникают вопросы сколько таких "матерей-героев" и не ошибка ли это
5. в столбце dob_years есть нулевые значения. Очень странно
6. days_employed и total_income будет удобнее представить в виде целых чисел  
Почти все вопросы возникают к качеству сбора данных, если с разным регистром можно смирится, то с пропущенными значениями надо что-то делать и дать обратную связь источнику данных.

## Шаг 2. Предобработка данных

### Обработка пропусков и некоректных данных

В данный блок добавил и обработку некоректных данных: дети, ну и у нас есть 3 проблемки: days_employed, total_income и нули в dob_years. 
Пропуски в days_employed и total_income - одни и те же? Проверим:

In [4]:
data[(data['total_income'].isnull() == True) & (data['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Действительно, одни и те же. Можно было просто заменить все на средние значения и работать дальше. Но давайте проверим в каких типах занятости есть нулевые значения и сделаем мини вывод:

In [5]:
data[(data['total_income'].isna() == True) & (data['days_employed'].isna() == True)]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

А теперь проверим - не зря ли это делали, посчитаем медиану по зп:

In [6]:
medians_total_income = data.groupby('income_type')['total_income'].median()
print(medians_total_income)

income_type
безработный       131,339.752
в декрете          53,829.131
госслужащий       150,447.935
компаньон         172,357.951
пенсионер         118,514.486
предприниматель   499,163.145
сотрудник         142,594.397
студент            98,201.625
Name: total_income, dtype: float64


Неплохо быть безработным) А медианная зп действительно отличается в зависимости от занятости. Не зря делали. Заменяем значения для каждого типа занятости:

In [7]:
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'госслужащий'), 'total_income'] = medians_total_income[2]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'компаньон'), 'total_income'] = medians_total_income[3]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'пенсионер'), 'total_income'] = medians_total_income[4]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'предприниматель'), 'total_income'] = medians_total_income[5]
data.loc[(data['total_income'].isna()) & (data['income_type'] == 'сотрудник'), 'total_income'] = medians_total_income[6]


In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Пропуски в total_income заполнены, а мы молодцы. Займемся столбцом days_employed, по тому же принципу: всем одинаковый стаж не поставить. Для начала сделаем все значения положительными и заменим значения

In [9]:
data['days_employed'] = data['days_employed'].apply(abs)
medians_days_employed = data.groupby('income_type')['days_employed'].median()
print(medians_days_employed)

income_type
безработный       366,413.653
в декрете           3,296.760
госслужащий         2,689.368
компаньон           1,547.382
пенсионер         365,213.306
предприниматель       520.848
сотрудник           1,574.203
студент               578.752
Name: days_employed, dtype: float64


Мы видим выбросы у безработных и пенсионеров. Гипотеза такая: их опыт посчитан в "часах", а не в "днях" (разделив их данные на 24 - получаются "нормальные" значения). Можно было бы написать функцию или цикл для исправления данного недочета, но для наших поставленных задач не требуется данный столбец, поэтому заполним данными какие есть и напишем источнику данных о проблеме с этими группами.

In [10]:
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'госслужащий'), 'days_employed'] = medians_days_employed[2]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'компаньон'), 'days_employed'] = medians_days_employed[3]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'пенсионер'), 'days_employed'] = medians_days_employed[4]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'предприниматель'), 'days_employed'] = medians_days_employed[5]
data.loc[(data['days_employed'].isna()) & (data['income_type'] == 'сотрудник'), 'days_employed'] = medians_days_employed[6]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [11]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,"8,437.673",42,высшее,0,женат / замужем,0,F,сотрудник,0,"253,875.639",покупка жилья
1,1,"4,024.804",36,среднее,1,женат / замужем,0,F,сотрудник,0,"112,080.014",приобретение автомобиля
2,0,"5,623.423",33,Среднее,1,женат / замужем,0,M,сотрудник,0,"145,885.952",покупка жилья
3,3,"4,124.747",32,среднее,1,женат / замужем,0,M,сотрудник,0,"267,628.550",дополнительное образование
4,0,"340,266.072",53,среднее,1,гражданский брак,1,F,пенсионер,0,"158,616.078",сыграть свадьбу
5,0,926.186,27,высшее,0,гражданский брак,1,M,компаньон,0,"255,763.565",покупка жилья
6,0,"2,879.202",43,высшее,0,женат / замужем,0,F,компаньон,0,"240,525.972",операции с жильем
7,0,152.780,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,"135,823.934",образование
8,2,"6,929.865",35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,"95,856.832",на проведение свадьбы
9,0,"2,188.756",41,среднее,1,женат / замужем,0,M,сотрудник,0,"144,425.938",покупка жилья для семьи


Остались еще вопросы по возрасту, у многих стоит 0. Узнаем сколько их и к какому income_type они относятся, затем заполним медианными значения по income_type.

In [12]:
data.loc[data['dob_years'] == 0, 'income_type'].value_counts()

сотрудник      55
компаньон      20
пенсионер      20
госслужащий     6
Name: income_type, dtype: int64

In [13]:
medians_dob_years = data.groupby('income_type')['dob_years'].median()
medians_dob_years

income_type
безработный       38.000
в декрете         39.000
госслужащий       40.000
компаньон         39.000
пенсионер         60.000
предприниматель   42.500
сотрудник         39.000
студент           22.000
Name: dob_years, dtype: float64

In [14]:
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'госслужащий'), 'dob_years'] = medians_dob_years[2]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'компаньон'), 'dob_years'] = medians_dob_years[3]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'пенсионер'), 'dob_years'] = medians_dob_years[4]
data.loc[(data['dob_years'] == 0) & (data['income_type'] == 'сотрудник'), 'dob_years'] = medians_dob_years[6]
data['dob_years'].describe()

count   21,525.000
mean        43.496
std         12.232
min         19.000
25%         34.000
50%         43.000
75%         53.000
max         75.000
Name: dob_years, dtype: float64

Займемся детьми, проверим сколько уникальных значений в этом столбце и количество каждого.

In [15]:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Ясно-понятно что отрицательное число детей не может быть, поэтому превращаем '-1' в '1' - возможно "человеческий фактор" пририсовал черточку, а так же - понимая логически и "гугля" - узнаем что в России всего 2 семьи с 20 детьми (Хромых и Шишкины), поэтому '20' детей у нас превращаются в '2', видимо потому что в NumPude 2 и 0 расположены рядом)

In [16]:
data['children'] = data['children'].replace(-1, 1)
data['children'] = data['children'].replace(20, 2)
data['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

Вот теперь выглядит красиво и больше похоже на правду, без "потерянных" детей и "мегасупергероей-матерей"

**Вывод**

Пропуски заполнены, так как они появлялись  не хаотично, а в определенных типах занятости - можно сделать предварительный вывод - человеческий фактор есть везде, даже в банках. Так же присутствует и техническая неисправность - пропуски в одних и тех же строках в 2 столбцах.

### Замена типа данных

Начнем с простого: поменяем float на int в столбцах days_employed и total_income. Будет выглядеть по-человечески. Применим astype, так как стаж и возраст можно исчислять целыми числами.

In [17]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


Заметим! после обработки столбца dob_years поменялся тип, поменяем его тоже.

In [18]:
data['dob_years'] = data['dob_years'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [19]:
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,152,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,6929,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


**Вывод**

Теперь у нас таблица "здорового" человека, все отлично читается

### Обработка дубликатов

Начнем с простого, удалим дубли вызванные разным регистром букв в столбце education. Сдемаем все строчными буквами и они сами уйдут. 
Это было ДО:

In [20]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

In [21]:
data['education'] = data['education'].str.lower()

А это **ПОСЛЕ**:

In [22]:
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Так же заоодно я бы проверил другие столбцы object, на всякий случай: 

In [23]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

Вопросов нет, идем дальше:

In [24]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

У нас появилось бесполое существо, сразу же понятно - удаление одной строки не повлияет на итоговый результат.

In [25]:
data = data.loc[data.loc[:, 'gender'] != 'XNA']

In [26]:
data['gender'].value_counts()

F    14236
M     7288
Name: gender, dtype: int64

Всё по классике, М и Ж

In [27]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5084
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Тут все ок.

Переидем к "взрослым" дубликатам. Посмотрим сколько дублей в нашем датасете с помощью duplicated():

In [28]:
data.duplicated().sum()

71

А теперь узнаем как они выглядят:

In [29]:
data[data.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1005,0,365213,62,среднее,1,женат / замужем,0,F,пенсионер,0,118514,ремонт жилью
1191,0,365213,61,среднее,1,женат / замужем,0,F,пенсионер,0,118514,операции с недвижимостью
1511,0,365213,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,118514,дополнительное образование
1681,0,365213,57,среднее,1,гражданский брак,1,F,пенсионер,0,118514,на проведение свадьбы
2052,0,365213,58,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
16148,0,1547,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба
17379,0,1547,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью
17774,1,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,строительство жилой недвижимости
19369,0,1547,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба


Теперь посчитаем: какой процент "бракованных" данных от общего количества строк, если меньше 1% - можно удалить их без дальнейшего разбирательства и "суда и следствия":

In [30]:
print('Процент дублей: {:.1%}'.format(data.duplicated().sum() / len(data)))

Процент дублей: 0.3%


Удаляем, ни о чем не жалея, и проверим сколько "их" осталось:

In [31]:
data = data.drop_duplicates()

In [32]:
data.duplicated().sum()

0

**Вывод**

Избавились от дублей, вызванных неправильным заполнением столбца education - тут поможет введение четких регламентов заполнения (только прописные буквы и т.д.), а так же маленький косячок в "половом" вопросе, не забываем про дубли, которые возможно появились из-за невнимательности или технического недочета. В итоге эти дубли не будут влиять на финальный результат наших исследований, надеюсь:)

### Лемматизация

Здесь мы узнаем на какие основные цели берутся кредиты, тут поможет столбец purpose. Сначала найдем все значения:

In [33]:
data['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка своего жилья                      620
покупка недвижимости                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Теперь немного магии, "лемантизируем" и увидим уникальные слова (и не только) в столбце purpose:

Получили список, выбираем популярное: пробелы, переносы и предлоги - не берем, оставляем только существительные. Остается: недвижимость, покупка, жилье, автомобиль, образование, операция, свадьба, строительство. Из этого модем убрать жилье, так как относится к недвижимости, покупку тоже убираем - подходит почти ко всему - а значит и нельзя назвать уникальным значением, по такой же логике "операция" удаляется. Итог: недвижимость, автомобиль, образование и свадьба.  С помощью lemmatization по столбцу purpose заменяем похожие по значению переменные на единые.

In [34]:
def lemmatization (purpose):
    
    lemma_row = m.lemmatize(purpose)
              
    if ('жилье' in lemma_row) or ('недвижимость' in lemma_row):
        lemma_name_purpose='недвижимость'
        return lemma_name_purpose
    elif 'автомобиль' in lemma_row:
        lemma_name_purpose='автомобиль'
        return lemma_name_purpose
    elif 'образование' in lemma_row:
        lemma_name_purpose='образование'
        return lemma_name_purpose
    else:
        lemma_name_purpose='свадьба'
        return lemma_name_purpose
    
purpose_lemma = data['purpose'].apply(lemmatization)
data['purpose'] = purpose_lemma
data.head(15)

data['purpose'].value_counts()

недвижимость    10810
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

**Вывод**

В столбце purpose теперь всего 4 значения, работать с ней легче теперь.

### Категоризация данных

Создадим несколько словарей, чтобы облегчить следующие задачи.

In [35]:
education_dict = data[['education_id', 'education']].drop_duplicates().reset_index(drop=True)
education_dict

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [36]:
family_dict = data[['family_status_id', 'family_status']].drop_duplicates().reset_index(drop=True)
family_dict

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,Не женат / не замужем


Для категоризации по доходу выведем командой describe данные, там мы увидим квантили (знаю что уже выводил, но листать на верх не удобно)

In [37]:
data['total_income'].describe()

count      21,453.000
mean      165,317.774
std        98,189.239
min        20,667.000
25%       107,620.000
50%       142,594.000
75%       195,818.000
max     2,265,604.000
Name: total_income, dtype: float64

Создадим функцию, которая добавит столбец с описанием уровня доходов, обзовем их "низкий", "средний", "выше среднего" и "высокий доход"

In [38]:
def total_income_quantile(row):
    if row < 107260:
        return 'низкий доход'
    if 107425 <= row < 142594:
        return 'средний доход'
    if 142594 <= row < 195818:
        return 'выше среднего доход'
    else:
        return 'высокий доход'
    
data['total_income_overall'] = data['total_income'].apply(total_income_quantile)
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_overall
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,высокий доход
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,средний доход
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,выше среднего доход
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,высокий доход
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,выше среднего доход


Применим общепринятую классификацию по возрастам, до 30 молодость, затем зрелость - обзовем ее взрослыми, и после 60 старость. Всё с помощью функции, подобной предыдущей задачи.

In [39]:
def dob_years_quantile(row):
    if row < 30:
        return 'молодые'
    if 30 <= row < 59:
        return 'взрослые'
    else:
        return 'старые'
    
data['dob_years_overall'] = data['dob_years'].apply(dob_years_quantile)
data.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_overall,dob_years_overall
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,высокий доход,взрослые
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,средний доход,взрослые
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,выше среднего доход,взрослые
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,высокий доход,взрослые
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,выше среднего доход,взрослые
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,недвижимость,высокий доход,молодые
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,недвижимость,высокий доход,взрослые
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,средний доход,взрослые
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,свадьба,низкий доход,взрослые
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,недвижимость,выше среднего доход,взрослые


**Вывод**

на данном этапе облегчили себе задачу, что бы в дальнейшем проще ответить на вопросы

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок? Сразу создадим функцию считающая процент просрочки кредитов и обзовем ее fail, она понадобится для итоговых выводов.

In [40]:
children_debt = data.pivot_table(index='children', columns='debt', aggfunc={'debt':'count'}).fillna(0).astype(int)

def fail(data):
    data['percentage'] = ((data[('debt', 1)] * 100) / (data[('debt', 0)] + data[('debt', 1)])).round(2)

fail(children_debt)
children_debt.sort_values(by='percentage', ascending=False)

debt       percentage
debt          0     1           
children                        
4            37     4      9.760
2          1926   202      9.490
1          4410   445      9.170
3           303    27      8.180
0         13027  1063      7.540
5             9     0      0.000

In [41]:
data_children = data.groupby('children').agg(count = ('debt','count'), debt_count = ('debt', 'sum'))
data_children['debt_part'] = (data_children['debt_count'] / data_children['count']) * 100
print(data_children.sort_values(by='debt_part', ascending=False))

          count  debt_count  debt_part
children                              
4            41           4      9.756
2          2128         202      9.492
1          4855         445      9.166
3           330          27      8.182
0         14090        1063      7.544
5             9           0      0.000


**Вывод**

Если у тебя 5 детей - залог хорошей кредитной истории) Шутка!  Вывод такой: данных по семьям с 4 и 5 детьми недостаточно, их слишком мало что бы сделать правильные выводы, но по остальным категориям можно подытожить: если у человека есть дети и кредит, в случае трудности он предпочтет детей, чем платить кредит. Но разница не критичная.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [43]:
family_status_debt = data.pivot_table(index='family_status_id', columns='debt', aggfunc={'debt':'count'}).merge(family_dict, on='family_status_id')

fail(family_status_debt)

family_status_debt.sort_values(by='percentage', ascending=False)

,family_status_id,"(debt, 0)","(debt, 1)",family_status,percentage
4,4,2536,274,Не женат / не замужем,9.750
1,1,3762,388,гражданский брак,9.350
0,0,11408,931,женат / замужем,7.550
3,3,1110,85,в разводе,7.110
2,2,896,63,вдовец / вдова,6.570


**Вывод**

Самый главный вывод - человек с опытом мудреет. Реально же выстроился жизненый путь: холост, нашел себе пару, женился, развелся или овдовел. Семейный статус действительно влияет на возврат кредита, но тут можно и притянуть возраст.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [44]:
income_debt = data.pivot_table(index='total_income_overall', 
                                                   columns='debt', 
                                                   aggfunc={'debt':'count'})
                                                                                              

fail(income_debt)
income_debt.sort_values(by='percentage', ascending=False)

debt      percentage
debt                     0    1           
total_income_overall                      
средний доход         4043  386      8.720
выше среднего доход   5770  546      8.640
низкий доход          4905  425      7.970
высокий доход         4994  384      7.140

**Вывод**

Очевидно что люди с самым высоким доходом оказаись в лидерах по возврату кредита, и неожиданно что люди с низкими доходами не сильно отстают от них. Средний доход похоже развращет человека, ему хочется больше, так же как и у след. группы - необязательные и импульсивные покупки могут испортить кредитную историю.

- Как разные цели кредита влияют на его возврат в срок?

In [45]:
purpose_debt = data.pivot_table(index='purpose', columns='debt', aggfunc={'debt':'count'})

fail(purpose_debt)

purpose_debt.sort_values(by='percentage', ascending=False)

debt      percentage
debt              0    1           
purpose                            
автомобиль     3903  403      9.360
образование    3643  370      9.220
свадьба        2138  186      8.000
недвижимость  10028  782      7.230

**Вывод**

Доступные автокредиты погубят человечество, возможно выбрал автомобиль по стоимости не соответствующей его доходам, еще и допов и страховок напихали туда. Образование - возможно берется молодыми людьми, которые должны учится и при этом платить. Патовая ситуация - учишься -> нет времени работать -> нечем платить, либо: работаешь -> платишь -> зачем учеба :). Свадьба - все хотят хорошую свадьбу, и видимо расчитвают на хорошие подарки - похоже получается. А недвижимость - очень осознаный шаг, да и срок кредта в таких сделках не маленький, ипотека и все такое, а так же - в России суровый климат, на улицу долго не выживешь - поэтому и платят до последнего.

## Шаг 4. Общий вывод

Теперь мы знаем идеальный портрет заемщика для банка: вдовец без детей с высоким доходом для покупки недвижимости.  
Получив "очень грязные" данные, обработали и получили ответы на все вопросы.  
Некорректность данных вызвана как человеческим фактором, так и техническим. Например: столбец education записан разным регистром - явно человеческий фактор, но вот наличие проблем в одних и тех же сроках в 2 столбцах явно намекает на какую-то закономерную ошибку. Отдельно надо сказать про стаж, в наших иследованиях он не нужен был, поэтому забили на него. Но он очень не корректен, скорее всего техническая проблема, которая у некоторых групп заемщиков переводила стаж из дней в часы.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.